<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 35px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Token Embedding
  </div> 
  
<div style="
      font-weight: normal; 
      font-size: 25px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Matrix Factorization methods
  </div> 



  <div style="
      font-size: 15px; 
      line-height: 12px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Jean-baptiste AUJOGUE
  </div> 


  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  December 2022
  </div>

<a id="TOC"></a>

#### Table Of Content

1. [Corpus](#data) <br>
2. [Token Embedding using SGD](#sgd) <br>
3. [Token Embedding using Matrix Factorization](#matrix) <br>

# Overview

We expose here pretraining methods for the _Token Embedding_ layer of a NLP model. The `transformers` library does not carry components for such pretraining, but it is still a valuable topic and was the center of many papers before transformer models and their contextual embeddings took the advantage.



The global purpose of Word Embedding is to represent a _Token_ , a raw string representing a unit of text, as a low dimensional (dense) vector. The way tokens are defined only depends on the method used to split a text into text units : using blank spaces as separators or using classical NLTK or SpaCy's segmentation models leave _words_ as tokens, but splitting protocols yielding _subword units_ , that are half-way between characters and full words, are also investigated :

- [Neural Machine Translation of Rare Words with Subword Units (2015)](https://www.aclweb.org/anthology/P16-1162.pdf)
- [Google’s Neural Machine Translation System: Bridging the Gap between Human and Machine Translation (2016)](https://arxiv.org/pdf/1609.08144.pdf)). 
- [BPEmb: Tokenization-free Pre-trained Subword Embeddings in 275 Languages (2018)](https://www.aclweb.org/anthology/L18-1473.pdf)
- [SentencePiece: A simple and language independent subword tokenizer and detokenizer for Neural Text Processing (2018)](https://arxiv.org/abs/1808.06226)


Here we broadly denote by _word_ any such token. Commonly followed approaches for the embedding of words (aka tokens) decompose into three levels of granularity :

| Level |  | |
|------|------|------|
| **Word** | [I.1 Custom model](#word_level_custom) | [I.2 Gensim Model](#gensim) |
| **sub-word unit** | [II.1 FastText model](#fastText) |  |
| **Character** |  |  |


<br>
Visualization with TensorBoard : https://www.tensorflow.org/guide/embedding (TODO)

# Training objectives

#### CBOW training objective

Cette méthode de vectorisation est introduite dans \cite{mikolov2013distributed, mikolov2013efficient}, et consiste à construire pour un vocabulaire de mots une table de vectorisation $T$ contenant un vecteur par mot. La spécificité de cette méthode est que cette vectorisation est faite de façon à pouvoir prédire chaque mot à partir de son contexte. La construction de cette table $T$ passe par la création d'un réseau de neurones, qui sert de modèle pour l'estimation de la probabilité de prédiction d'un mot $w_t$ d'après son contexte $c = w_{t-N}, \, ... \, , w_{t-1}$, $w_{t+1}, \, ... \, , w_{t+N}$. La table $T$ intégrée au modèle sera optimisée lorsque ce modèle sera entrainé de façon à ce qu'un mot $w_t$ maximise la vraisemblance de la probabilité $P(. \, | \, c)$ fournie par le modèle. 

Le réseau de neurones de décrit de la façon suivante :

![cbow](figs/CBOW.png)

Un contexte $c = w_{t-N}, \, ... \, , w_{t-1}$, $w_{t+1}, \, ... \, , w_{t+N}$ est vectorisé via une table $T$ fournissant un ensemble de vecteurs denses (typiquement de dimension comprise entre 50 et 300) $T(w_{t-N}), \, ... \, , T(w_{t-1})$, $T(w_{t+1}), \, ... \, , T(w_{t+N})$. Chaque vecteur est ensuite transformé via une transformation affine, dont les vecteurs résultants sont superposés en un unique vecteur

\begin{align*}
v_c = \sum _{i = - N}^N M_i T(w_{t+i}) + b_i
\end{align*}

Le vecteur $v_c$ est de dimension typiquement égale à la dimension de la vectorisation de mots. Une autre table $T'$ est utilisée pour une nouvelle vectorisation du vocabulaire, de sorte que le mot $w_{t}$ soit transformé en un vecteur $T'(w_{t})$ par cette table, et soit proposé en position $t$ avec probabilité

\begin{align*}
P(w_{t} \, | \, c\,) = \frac{\exp\left( T'(w_{t}) \cdot v_c \right) }{\displaystyle \sum _{w \in \mathcal{V}} \exp\left(   T'(w) \cdot v_c 
\right) }
\end{align*}

Ici $\cdot$ désigne le produit scalaire entre vecteurs. L'optimisation de ce modèle permet d'ajuster la table $T$ afin que les vecteurs de mots portent suffisamment d'information pour reformer un mot à partir du contexte.


#### Skip-Gram training objective


Cette méthode de vectorisation est introduite dans \cite{mikolov2013distributed, mikolov2013efficient} comme version mirroir au Continuous Bag Of Words, et consiste là encore à construire pour un vocabulaire de mots une table de vectorisation $T$ contenant un vecteur par mot. La spécificité de cette méthode est que cette vectorisation est faite non pas de façon prédire un mot central $w$ à partir d'un contexte $c $ comme pour CBOW, mais plutôt de prédire le contexte $c $ à partir du mot central $w$. La construction de cette table $T$ passe par la création d'un réseau de neurones servant de modèle pour l'estimation de la probabilité de prédiction d'un contexte $c = w_{t-N}, \, ... \, , w_{t-1}$, $w_{t+1}, \, ... \, , w_{t+N}$ à partir d'un mot central $w_t$. La table $T$ intégrée au modèle sera optimisée lorsque ce modèle sera entrainé de façon à ce que le contexte  $ c $ maximise la vraisemblance de la probabilité $P( . \, | \, w_t)$ fournie par le modèle.


Une implémentation de ce modèle est la suivante : 


![skipgram](figs/Skipgram.png)


Un mot courant $w_t$ est vectorisé par une table $T$ fournissant un vecteur dense (typiquement de dimension comprise entre 50 et 300) $T(w_t)$. Ce vecteur est alors transformé en un ensemble de $2N$ vecteurs

\begin{align*}
\sigma (M_{i} T(w_t) + b_{i}) \qquad \qquad i =-N,\, ...\, , -1, 1, \, ...\, , N
\end{align*}

où $N$ désigne la taille de la fenêtre retenue, d'une dimension typiquement égale à la dimension de la vectorisation de mots, et $\sigma$ une fonction non linéaire (typiquement la _Rectified Linear Unit_ $\sigma (x) = max (0, x)$). Une autre table $T'$ est utilisée pour une nouvelle vectorisation du vocabulaire, de sorte que chaque mot $w_{t+i}$, transformé en un vecteur $T'(w_{t+i})$ par cette table, soit proposé en position $t+i$ avec probabilité

\begin{align*}
P( w_{t+i} | \, w_t) = \frac{\exp\left(  T'(w_{t+i}) ^\perp \sigma \left( M_i T(w_t) + b_{i}\right) \right) }{\displaystyle \sum _{w \in \mathcal{V}} \exp\left(   T'(w) ^\perp \sigma \left( M_i T(w_t) + b_i\right) \right) }
\end{align*}

On modélise alors la probabilité qu'un ensemble de mots $c = w_{t-N}, \, ... \, , w_{t-1}$, $w_{t+1}, \, ... \, , w_{t+N}$ soit le contexte d'un mot $w_t$ par le produit

\begin{align*}
 P( c\, | \, w_t) = \prod _{i = -N}^N P( w_{t+i}\, | \, w_t)
\end{align*}

Ce modèle de probabilité du contexte d'un mot est naif au sens où les mots de contextes sont considérés comme indépendants deux à deux dès lors que le mot central est connu. Cette approximation rend cependant le calcul d'optimisation beaucoup plus court.



L'optimisation de ce modèle permet d'ajuster la table $T$ afin que les vecteurs de mots portent suffisamment d'information pour reformer l'intégralité du contexte à partir de ce seul mot. La vectorisation Skip-Gram est typiquement plus performante que CBOW, car la table $T$ subit plus de contrainte dans son optimisation, et puisque le vecteur d'un mot est obtenu de façon à pouvoir prédire l'utilisation réelle du mot, ici donnée par son contexte. 


A complete review of methods for learning Token Embeddings is provided in this [PhD thesis, 2018](https://www.skoltech.ru/app/data/uploads/2018/09/Thesis-Fonarev1.pdf).

# Packages

[Back to top](#plan)

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import os
import time
import math
import re
import random
import pickle
import copy
from unidecode import unidecode
import multiprocessing

# data 
import numpy as np
import pandas as pd
from datasets import Dataset, load_from_disk

# models
from transformers import AutoTokenizer
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from mittens import GloVe

# viz
from tqdm import tqdm

#### Custom paths & imports

In [5]:
path_to_repo = os.path.dirname(os.getcwd())
path_to_data = os.path.join(path_to_repo, 'datasets', 'clinical trials CTTI')
path_to_save = os.path.join(path_to_repo, 'saves', 'MLM')
path_to_src  = os.path.join(path_to_repo, 'src')

#### Constants

In [6]:
dataset_name = 'clinical-trials-ctti'
base_model_name = os.path.join('albert-small-clinical-trials', 'tokenizer')
final_model_name = os.path.join('albert-small-clinical-trials', 'w2v')

<a id="data"></a>

# 1. Corpus

[Table of content](#TOC)

## 1.1 Load Clinical Trials corpus

[Table of content](#TOC)

In [7]:
with open(os.path.join(path_to_data, '{}.txt'.format(dataset_name)), 'r', encoding = 'utf-8') as f:
    texts = f.readlines()

In [8]:
len(texts)

3464685

## 1.2 Tokenize corpus

[Table of content](#TOC)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(os.path.join(path_to_save, base_model_name))

Create & export tokenized corpus (uncoment and run this only once):

In [7]:
# tokenized_corpus = [tokenizer.tokenize(t) for t in tqdm(texts)]
# dataset = Dataset.from_dict({'text': tokenized_corpus})
# dataset.save_to_disk(os.path.join(path_to_data, 'tmp', '{}-w2v'.format(dataset_name)))

Import back tokenized corpus :

In [10]:
dataset = load_from_disk(os.path.join(path_to_data, 'tmp', '{}-w2v'.format(dataset_name)))

In [8]:
tokenized_corpus = [dataset[i]['text'] for i in tqdm(range(len(dataset)))]

100%|██████████████████████████████████████████████████████████████████████| 3464685/3464685 [09:56<00:00, 5810.84it/s]


In [25]:
tokenized_corpus[0]

['▁this',
 '▁study',
 '▁will',
 '▁test',
 '▁the',
 '▁ability',
 '▁of',
 '▁extended',
 '▁release',
 '▁',
 'n',
 'i',
 'f',
 'ed',
 'i',
 'pine',
 '▁(p',
 'roc',
 'ard',
 'i',
 'a',
 '▁',
 'x',
 'l',
 '),',
 '▁',
 'a',
 '▁blood',
 '▁pressure',
 '▁medication',
 ',',
 '▁to',
 '▁per',
 'mit',
 '▁',
 'a',
 '▁decrease',
 '▁in',
 '▁the',
 '▁dose',
 '▁of',
 '▁glucocorticoid',
 '▁medication',
 '▁children',
 '▁take',
 '▁to',
 '▁treat',
 '▁congenital',
 '▁adrenal',
 '▁hyper',
 'plas',
 'i',
 'a',
 '▁(c',
 'a',
 'h',
 ').']

<a id="matrix"></a>


# 2. Token Embedding using Matrix Factorization

[Table of content](#TOC)

The comparison between SGD-based learning and matrix factorization approaches for token embeddings is developped in [Neural Word Embedding
as Implicit Matrix Factorization (2014)](https://papers.nips.cc/paper/2014/file/feab05aa91085b7a8012516bc3533958-Paper.pdf).


The paper [Glove (2015)](https://nlp.stanford.edu/pubs/glove.pdf)

Opensource implementations :
- original Stanford's [Glove](https://nlp.stanford.edu/projects/glove/), written in C.
- [glove-python](https://github.com/maciejkula/glove-python), which fails to build and is hence discarded.
- [mittens](https://github.com/roamanalytics/mittens)
- various pure python implementations : [here](https://gist.github.com/emaadmanzoor/1d06e0751a3f7d39bc6814941b37531d)

In [15]:
import scipy
import itertools

Token-token distance matrix:

For a token _w_ of the vocabulary, and a tokenized text _t_ of the corpus, denote by t[w] the list of indexes of occurences of w in t.

$$ M_{i, j} = \frac{\displaystyle \sum_{t \in T}\sum_{m \in t[w_i]}\sum_{n \in t[w_j]} d(m, n)}{\displaystyle \left( \sum_{t \in T}\#t[w_i] \right) \left( \sum_{t \in T}\# t[w_j] \right)}$$

Where typical choices of function _d_ will be 

$$ d_r(a, b) = 1 \text{ if } \lvert a - b \rvert \leqslant r \text{ else } 0 \;\; \text{ (Hard window)} \qquad \qquad  d(a, b) = \frac{1}{1 + \lvert a - b \rvert} \;\; \text{(} L^1 \text{-decrease)}$$


In [128]:
class lebesgue_decrease:
    def __init__(self, p = 1):
        self.p = p

    def __call__(self, m, n):
        return 1/(1+abs(m-n))**self.p



def get_kernel(size, weight_fct, framework = 'scipy'):
    # using list comprehension
    k = np.array([[weight_fct(i, j) for j in range(size)] for i in range(size)], dtype = 'float32')
    if framework == 'scipy':
        k = scipy.sparse.csc_matrix(k)
    return k



def onehot_unfold(indices, n_tokens, framework = 'scipy'):
    N = indices.size
    if framework == 'scipy':
        data = np.ones(N, dtype = int)
        v = scipy.sparse.csc_matrix((data, (np.arange(N), indices.ravel())), shape = (N, n_tokens))
    elif framework == 'numpy':
        v = np.zeros((N, n_tokens))
        v[np.arange(N), indices] = 1
    else:
        raise NotImplementedError('"framework" should be "scipy" or "numpy"')
    return v



def build_co_occurrence_matrix_from_tokens(tokenized_corpus, token2id, weight_fct):
    sim_matrix = scipy.sparse.identity(len(token2id), format = 'lil')
    for t in tqdm(tokenized_corpus):
        for m, n in itertools.permutations(range(len(t)), r = 2):
            sim_matrix[token2id[t[m]], token2id[t[n]]] += weight_fct(m, n)
    return sim_matrix



def build_co_occurrence_matrix_from_indices(indexed_corpus, n_tokens, weight_fct):
    sim_matrix = scipy.sparse.identity(n_tokens, format = 'lil')
    for t in tqdm(indexed_corpus):
        for m, n in itertools.permutations(range(len(t)), r = 2):
            sim_matrix[t[m], t[n]] += weight_fct(m, n)
    return sim_matrix



# TODOs:
# - process by batch
# - move sparse tensor operations on gpu with pytorch ?
def build_co_occurrence_matrix_from_np_tensors(np_corpus, max_length, n_tokens, weight_fct, framework = 'scipy'):
    # init co-occurence matrix
    if framework == 'scipy':
        m = scipy.sparse.csc_matrix((n_tokens, n_tokens), dtype = 'float32')
    elif framework == 'numpy':
        m = np.zeros((n_tokens, n_tokens), dtype = 'float32')
    else:
        raise NotImplementedError('"framework" should be "scipy" or "numpy"')
    
    # fill co-occurence matrix
    k = get_kernel(max_length, weight_fct, framework) # (N, N)
    for t in tqdm(np_corpus):
        v = onehot_unfold(t, n_tokens, framework)     # (N, n_tokens)
        s = v.T @ k @ v                               # (n_tokens, n_tokens)
        m += s                                        # (n_tokens, n_tokens)
    return m

In [91]:
t = np.array([3, 1])
n_tokens = 4
s = scipy.sparse.identity(n_tokens, format = 'lil')

In [92]:
lol1 = init_similarity_matrix_sparse(len(t), lebesgue_decrease(0.75)) # (N, N)
lol2 = onehot_sparse(t, n_tokens) # (N, n_tokens)


s += lol2.T @ lol1 @ lol2


lol1.todense(), lol2.todense(), s.todense()

In [119]:
# build_co_occurrence_matrix_from_tokens(dataset[:100]['text'], token2id = tokenizer.get_vocab(), weight_fct = lebesgue_decrease())

In [120]:
# lol = [tokenizer(t, return_token_type_ids = False, return_attention_mask = False)['input_ids'][1:-1] for t in texts[:100]]

# build_co_occurrence_matrix_from_indices(lol, n_tokens = len(tokenizer.get_vocab()), weight_fct = lebesgue_decrease())

In [130]:
lol = [
    tokenizer(
        t, 
        return_token_type_ids = False, 
        return_attention_mask = False, 
        padding = 'max_length',
        truncation = True,
        max_length = 256+2,
        return_tensors = 'np',
    )['input_ids'][0][1:-1] 
    for t in texts[:10]
]

build_co_occurrence_matrix_from_np_tensors(
    lol, 
    max_length = 256, 
    n_tokens = len(tokenizer.get_vocab()), 
    weight_fct = lebesgue_decrease(),
    framework = 'scipy',
)

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 322.78it/s]


matrix([[13252.5841674,     0.       ,     0.       , ...,     0.       ,
             0.       ,     0.       ],
        [    0.       ,     0.       ,     0.       , ...,     0.       ,
             0.       ,     0.       ],
        [    0.       ,     0.       ,     0.       , ...,     0.       ,
             0.       ,     0.       ],
        ...,
        [    0.       ,     0.       ,     0.       , ...,     0.       ,
             0.       ,     0.       ],
        [    0.       ,     0.       ,     0.       , ...,     0.       ,
             0.       ,     0.       ],
        [    0.       ,     0.       ,     0.       , ...,     0.       ,
             0.       ,     0.       ]])